In [ ]:
from IPython.display import display, HTML
import os
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%run ../nb_core/root_dirs.ipynb
setup_syspath_disentangle(False)
%run ../nb_core/disentangle_imports.ipynb

In [ ]:
ckpt_dir = "/group/jug/ashesh/training/disentangle/2406/D25-M3-S0-L8/10"
assert os.path.exists(ckpt_dir)

In [ ]:
def get_dtype(ckpt_fpath):
    if os.path.isdir(ckpt_fpath):
        ckpt_fpath = ckpt_fpath[:-1] if ckpt_fpath[-1] == '/' else ckpt_fpath
    elif os.path.isfile(ckpt_fpath):
        ckpt_fpath = os.path.dirname(ckpt_fpath)
    assert ckpt_fpath[-1] != '/'
    return int(ckpt_fpath.split('/')[-2].split('-')[0][1:])

In [ ]:
dtype = get_dtype(ckpt_dir)
dtype

In [ ]:
from disentangle.data_loader.patch_index_manager import TilingMode

image_size_for_grid_centers = 32
mmse_count = 10
custom_image_size = None
data_t_list = None # [0]
tiling_mode = TilingMode.ShiftBoundary

save_comparative_plots =False
enable_calibration = False
batch_size = 16
num_workers = 4
COMPUTE_LOSS = False
use_deterministic_grid = None
threshold = None # 0.02
compute_kl_loss = False
evaluate_train = False# inspect training performance
eval_datasplit_type = DataSplitType.Test 
val_repeat_factor = None
psnr_type = 'range_invariant' #'simple', 'range_invariant'

In [ ]:
%run ../nb_core/config_loader.ipynb

In [ ]:
tokens = ckpt_dir.split('/')
idx = tokens.index('disentangle')
if config.model.model_type == 25 and tokens[idx+1] == '2312':
    config.model.model_type = ModelType.LadderVAERestrictedReconstruction

In [ ]:
from disentangle.core.sampler_type import SamplerType
from disentangle.core.loss_type import LossType
from disentangle.data_loader.ht_iba1_ki67_rawdata_loader import SubDsetType
# from disentangle.core.lowres_merge_type import LowresMergeType


with config.unlocked():
    if 'depth3D' in config.data and config.data.depth3D > 1:
        config.data.mode_3D = True
        config.model.mode_3D = True
        
    config.model.skip_nboundary_pixels_from_loss = None
    if config.model.model_type == ModelType.UNet and 'n_levels' not in config.model:
        config.model.n_levels = 4
    if config.data.sampler_type == SamplerType.NeighborSampler:
        config.data.sampler_type = SamplerType.DefaultSampler
        config.loss.loss_type = LossType.Elbo
        config.data.grid_size = config.data.image_size
    if 'ch1_fpath_list' in config.data:
        config.data.ch1_fpath_list = config.data.ch1_fpath_list[:1]
        config.data.mix_fpath_list = config.data.mix_fpath_list[:1]
    if config.data.data_type == DataType.Pavia2VanillaSplitting:
        if 'channel_2_downscale_factor' not in config.data:
            config.data.channel_2_downscale_factor = 1
    if config.model.model_type == ModelType.UNet and 'init_channel_count' not in config.model:
        config.model.init_channel_count = 64
    
    if 'skip_receptive_field_loss_tokens' not in config.loss:
        config.loss.skip_receptive_field_loss_tokens = []
    
    if dtype == DataType.HTIba1Ki67:
        config.data.subdset_type = SubDsetType.OnlyIba1P30
        config.data.empty_patch_replacement_enabled = False
    
    if 'lowres_merge_type' not in config.model.encoder:
        config.model.encoder.lowres_merge_type = 0
    if 'validtarget_random_fraction' in config.data:
        config.data.validtarget_random_fraction = None
    
    if config.data.data_type == DataType.TwoDset:
        config.model.model_type = ModelType.LadderVae
        for key in config.data.dset1:
            config.data[key] = config.data.dset1[key]
    
    if 'dump_kth_frame_prediction' in config.training:
        config.training.dump_kth_frame_prediction = None

    if 'input_is_sum' not in config.data:
        config.data.input_is_sum = False

In [ ]:
dtype = config.data.data_type

if dtype in [DataType.CustomSinosoid, DataType.CustomSinosoidThreeCurve]:
    data_dir = f'{DATA_ROOT}/sinosoid_without_test/sinosoid/'
elif dtype == DataType.OptiMEM100_014:
    data_dir = f'{DATA_ROOT}/microscopy/'
elif dtype == DataType.Prevedel_EMBL:
    data_dir = f'{DATA_ROOT}/Prevedel_EMBL/PKG_3P_dualcolor_stacks/NoAverage_NoRegistration/'
elif dtype == DataType.AllenCellMito:
    data_dir = f'{DATA_ROOT}/allencell/2017_03_08_Struct_First_Pass_Seg/AICS-11/'
elif dtype == DataType.SeparateTiffData:
    data_dir = f'{DATA_ROOT}/ventura_gigascience'
elif dtype == DataType.SemiSupBloodVesselsEMBL:
    data_dir = f'{DATA_ROOT}/EMBL_halfsupervised/Demixing_3P'
elif dtype == DataType.Pavia2VanillaSplitting:
    data_dir = f'{DATA_ROOT}/pavia2'
# elif dtype == DataType.ExpansionMicroscopyMitoTub:
    # data_dir = f'{DATA_ROOT}/expansion_microscopy_Nick/'
elif dtype == DataType.ShroffMitoEr:
    data_dir = f'{DATA_ROOT}/shrofflab/'
elif dtype == DataType.HTIba1Ki67:
    data_dir = f'{DATA_ROOT}/Stefania/20230327_Ki67_and_Iba1_trainingdata/'
elif dtype == DataType.BioSR_MRC:
    data_dir = f'{DATA_ROOT}/BioSR/'
elif dtype == DataType.ExpMicroscopyV2:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v2/datafiles/'
elif dtype == DataType.TavernaSox2GolgiV2:
    data_dir = f'{DATA_ROOT}/TavernaSox2Golgi/acquisition2/'
elif dtype == DataType.Pavia3SeqData:
    data_dir = f'{DATA_ROOT}/pavia3_sequential/'
elif dtype == DataType.NicolaData:
    data_dir = f'{DATA_ROOT}/nikola_data/20240531/'
elif dtype == DataType.Dao3ChannelWithInput:
    data_dir = f'{DATA_ROOT}/Dao4Channel/'
elif dtype == DataType.Dao3Channel:
    data_dir = f'{DATA_ROOT}/Dao3Channel/'
elif dtype == DataType.SilvioLabCSHLData:
    data_dir = f'{DATA_ROOT}/svilen_cshl2024/'
elif dtype == DataType.ExpMicroscopyV3:
    data_dir = f'{DATA_ROOT}/expansion_microscopy_v4/405_NHS_488BODIPY/'
elif dtype == DataType.Elisa3DData:
    data_dir = f'{DATA_ROOT}/Elisa3D/'

In [ ]:
# config.model.noise_model_ch1_fpath = config.model.noise_model_ch1_fpath.replace('/home/ashesh.ashesh/training/', '/group/jug/ashesh/training_pre_eccv/')
# config.model.noise_model_ch2_fpath = config.model.noise_model_ch2_fpath.replace('/home/ashesh.ashesh/training/', '/group/jug/ashesh/training_pre_eccv/')

In [ ]:
%run ../nb_core/disentangle_setup.ipynb

In [ ]:
from disentangle.data_loader.nikola_7D_rawdata_loader import NikolaChannelList
import copy
config_other = copy.deepcopy(config)
with config_other.unlocked():
    config_other.data.channel_idx_list = [NikolaChannelList.Ch_D, NikolaChannelList.Ch_B, NikolaChannelList.Ch_BD]

_, val_dset_BT = create_dataset(config_other, data_dir, eval_datasplit_type=eval_datasplit_type,
                                      kwargs_dict=dloader_kwargs)

In [ ]:
_,ax = plt.subplots(figsize=(6,3),ncols=2)
ax[0].imshow(val_dset._data[0,:400,:400,2])
ax[1].imshow(val_dset_BT._data[0,:400,:400,2])

In [ ]:
if config.data.multiscale_lowres_count is not None and custom_image_size is not None:
    model.reset_for_different_output_size(custom_image_size)

In [ ]:
if data_t_list is not None:
    val_dset.reduce_data(t_list=data_t_list)
    val_dset_BT.reduce_data(t_list=data_t_list)

In [ ]:
from disentangle.nets.epistemic_uncertainty import enable_epistemic_uncertainty_computation_mode
test_input = torch.Tensor(val_dset[0][0][None]).cuda()
enable_epistemic_uncertainty_computation_mode(model, inp=test_input)

## Bleedthrough Dataset

In [ ]:
class BleedthroughDataset:
    def __init__(self, primary_dset, bleedthrough_dset, bleedthrough_weight):
        self.primar_dset = primary_dset
        self.bleedthrough_dset = bleedthrough_dset
        self.w = bleedthrough_weight
        assert len(primary_dset) == len(bleedthrough_dset)
    
    def __len__(self):
        return len(self.primar_dset)
    
    def __getitem__(self, idx):
        inp, tar = self.primar_dset[idx]
        inp_BT, _ = self.bleedthrough_dset[idx]

        inp = inp * (1-self.w) + inp_BT * self.w
        return inp, tar

In [ ]:
from disentangle.analysis.stitch_prediction import stitch_predictions
from disentangle.analysis.mmse_prediction import get_dset_predictions
# from disentangle.analysis.stitch_prediction import get_predictions as get_dset_predictions
std_dict = {}

for bt_weight in [0, 0.2, 0.5, 0.9]:
    dset_bt = BleedthroughDataset(val_dset, val_dset_BT, bt_weight)
    pred_tiled, rec_loss, logvar_tiled, patch_psnr_tuple, pred_std_tiled = get_dset_predictions(model, 
                                                    dset_bt, batch_size,
                                                num_workers=num_workers,
                                                mmse_count=mmse_count,
                                                model_type = config.model.model_type,
                                                )
    tmp = np.round([x.item() for x in patch_psnr_tuple],2)
    print('Patch wise PSNR, as computed during training', tmp,np.mean(tmp))
    std_dict[bt_weight] = pred_std_tiled

In [ ]:
for key in std_dict.keys():
    std_dict[key] = stitch_predictions(std_dict[key],val_dset)

In [ ]:
# for key in std_dict.keys():
#     std_dict[key] = std_dict[key][...,:2]

In [ ]:
import seaborn as sns
save_to_file = True

_,ax = plt.subplots(figsize=(9,4),ncols=2)
for ch_idx in [0,1]:
    for bt_weight in [0, 0.2, 0.5, 0.9]:
        std = std_dict[bt_weight][...,ch_idx]
        sns.kdeplot(std[:,::4,::4].reshape(-1,), log_scale=True, label=f'BT weight={bt_weight}', ax=ax[ch_idx])

# enable legend in ax[0]
ax[0].legend()
ax[1].legend()

if save_to_file:
    new_data = '.' .join(map(str, config_other.data.channel_idx_list))
    model_id = '_'.join(ckpt_dir.strip('/').split('/')[-3:])
    plot_fpath = os.path.join('/group/jug/ashesh/naturemethods/epistemic_uncertainty', f'OOD_similar_structure_newdata{new_data}_kdeplot_{model_id}.png')
    print(plot_fpath)
    plt.savefig(plot_fpath, dpi=200)


In [ ]:
from disentangle.analysis.plot_utils import clean_ax
save_to_file = True

keys = list(std_dict.keys())
_,ax = plt.subplots(figsize=(3*len(keys),3*2),ncols=len(keys),nrows=2)
for ch_idx in [0,1]:
    vmin = 10000 
    vmax = 0 
    for key in std_dict.keys():
        vmin = min(vmin, std_dict[key][...,ch_idx].min())
        vmax = max(vmax, std_dict[key][...,ch_idx].max())
    
    for i,key in enumerate(keys):
        ax[ch_idx,i].imshow(std_dict[key][0,...,ch_idx],vmin=vmin,vmax=vmax)
        if ch_idx == 0:
            ax[ch_idx,i].set_title(f'BT weight={key}')
        

clean_ax(ax)
# plt.tight_layout()
# reduce the space between the plots
plt.subplots_adjust(wspace=0.03, hspace=0.03)

if save_to_file:
    new_data = '.' .join(map(str, config_other.data.channel_idx_list))
    model_id = '_'.join(ckpt_dir.strip('/').split('/')[-3:])
    plot_fpath = os.path.join('/group/jug/ashesh/naturemethods/epistemic_uncertainty', f'OOD_similar_structure_newdata{new_data}_uncertainty_{model_id}.png')
    print(plot_fpath)
    plt.savefig(plot_fpath, dpi=200)


In [ ]:
from disentangle.analysis.plot_utils import clean_ax, add_subplot_axes

inset_min_labelsize=10

def add_inset(ax_, img_crop, inset_rect, vmin=None, vmax=None):
    inset_ax = add_subplot_axes(ax_, inset_rect, min_labelsize=inset_min_labelsize)
    inset_ax.imshow(img_crop, vmin=vmin, vmax=vmax)
    for spine in inset_ax.spines.values():
            spine.set_edgecolor('white')
    clean_ax(inset_ax)



In [ ]:
import matplotlib.patches as patches
_,ax = plt.subplots(figsize=(3*len(keys),3),ncols=len(keys),nrows=1)
# vmin = 10000
# vmax = 0
# for i,w in enumerate(keys):
#     inp_bt = val_dset_BT._data[...,-1]*w + val_dset._data[...,-1]*(1-w)
#     vmin = min(vmin, inp_bt.min())
#     vmax = max(vmax, inp_bt.max())
h = 1250
w = 500
sz = 300
inset_rect=[0.6,0.55,0.5,0.5]
rect_offset = 0.04

for i,weight in enumerate(keys):
    inp_bt = val_dset_BT._data[...,-1]*weight + val_dset._data[...,-1]*(1-weight)
    # ax[i].imshow(inp_bt[0,:300,:300])
    ax[i].imshow(inp_bt[0])
    # draw a white rectangle
    rect = patches.Rectangle((w,h),sz,sz,linewidth=1,edgecolor='w',facecolor='none')
    ax[i].add_patch(rect)
    new_offset = [inset_rect[0]-i*rect_offset, inset_rect[1], inset_rect[2], inset_rect[3]]
    add_inset(ax[i], inp_bt[0,h:h+sz,w:w+sz], new_offset)

clean_ax(ax)
plt.subplots_adjust(wspace=0.03, hspace=0.03)
if save_to_file:
    new_data = '.' .join(map(str, config_other.data.channel_idx_list))
    model_id = '_'.join(ckpt_dir.strip('/').split('/')[-3:])
    plot_fpath = os.path.join('/group/jug/ashesh/naturemethods/epistemic_uncertainty', f'OOD_similar_structure_newdata{new_data}_input_{model_id}.png')
    print(plot_fpath)
    plt.savefig(plot_fpath, dpi=200)
